In [1]:
from flask import Flask,jsonify
from flask import request
import re
import string 
import pandas as pd
import numpy as np
import tensorflow as tf
from nltk.tokenize import word_tokenize
from bs4 import BeautifulSoup
from nlp_id.postag import PosTag
#import fasttext.util
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.models import Model
from tensorboard.plugins import projector
from tensorflow import keras
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from bs4 import BeautifulSoup
from flask_cors import CORS

factory = StopWordRemoverFactory()
stopword = factory.create_stop_word_remover()
factory_stemming = StemmerFactory()
stemmer = factory_stemming.create_stemmer()
app = Flask(__name__)
#model machine learning download di 
#https://drive.google.com/file/d/1wl796Qa9LhE7dhQFIcSNx_A1gIQsKMCK/view?usp=sharing
model = keras.models.load_model("E:/MODEL MACHINE LEARNING/LSTM-RNN/trial_error/model-lstm-1-rnn-150-v1")
postagger = PosTag()
def remove_punct(text):
    text_nonpunct = ''
    text_nonpunct = re.sub('['+string.punctuation+']','',text)
    return text_nonpunct

def lower_token(tokens):
    return [w.lower() for w in tokens]

def cleanParagraph(paragraph):
    paragraph_stopword_removed = stopword.remove(paragraph)
    text_steammed = stemmer.stem(paragraph_stopword_removed)
    return text_steammed

def generateCleanText(lists):
    clean_text = []
    length_data = len(lists['content'])
    for i in range(0,length_data):
        lists['content'][i] = BeautifulSoup(lists['content'][i], "lxml").text
        lists['content'][i] = lists['content'][i].replace('Liputan6.com, ','').replace('JAKARTA,KOMPAS.com','').replace('JAKARTA, KOMPAS.com','').replace('Liputan.com, Jakarta','')
        lists['content'][i] = lists['content'][i].replace('kompas com','')
        lists['content'][i] = re.sub(r"\d+","",lists['content'][i])
        lists['content'][i] = re.sub('[^a-zA-Z]', ' ',lists['content'][i])
        lists['content'][i] = lists['content'][i].replace('\n','')
        lists['content'][i] = lists['content'][i].lower()
        clean_text.append(cleanParagraph(lists['content'][i]))
    return clean_text

def tokenizing(listData):
    tokens = []
    length_data = len(listData['text_clean'])
    for i in range(0,length_data):
        tokens.append(postagger.get_pos_tag(listData['text_clean'][i]))
        listData['text_clean'][i] = remove_punct(listData['text_clean'][i])
    return tokens

Using TensorFlow backend.


In [2]:
d_train = pd.read_csv('data/data_train.csv')
d_train['tokens'] = tokenizing(d_train)
d_test = pd.read_csv('data/data_test.csv')
d_test['tokens'] = tokenizing(d_test)
training_words = [word for tokens in d_train["tokens"] for word in tokens]
testing_words = [word for tokens in d_test["tokens"] for word in tokens]
all_training_words = training_words + testing_words
training_sentence_lengths = [len(tokens) for tokens in d_train["tokens"]]
TRAINING_VOCAB = sorted(list(set(all_training_words)))

tokenizer = Tokenizer(num_words=len(TRAINING_VOCAB),filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~',lower=True,char_level=False)
tokenizer.fit_on_texts(d_train["text_clean"].values)

e:\anaconda\envs\tf\lib\site-packages\ipykernel_launcher.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
CORS(app)
@app.route('/',methods=['POST'])
def predict():
    MAX_SEQUENCE_LENGTH = 150
    param = request.form.get('text')
    data = []
    data.append(param)
    data_predict = pd.DataFrame(data,columns=['content'])
    data_predict['content'][0] = remove_punct(data_predict['content'][0])
    data_predict['content'][0] = re.sub(r"\d+","",data_predict['content'][0])
    data_predict['content'][0] = re.sub('[^a-zA-Z]', ' ',data_predict['content'][0])
    data_predict['content'][0] = data_predict['content'][0].lower()
    data_predict['content'][0] = data_predict['content'][0].replace('\n','')
    data_predict_remove_stopword = stopword.remove(data_predict['content'][0])
    data_predict['clean_text'] = stemmer.stem(data_predict_remove_stopword)
    print(data_predict['clean_text'][0])
    test_sequences = tokenizer.texts_to_sequences(data_predict['clean_text'].values)
    print(test_sequences)
    data_predict_pad = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)
    
    result = model.predict(data_predict_pad.reshape(1,data_predict_pad.shape[1]),batch_size=128)[0]
    result_predicted = np.argmax(result)
    if result_predicted == 0 :
        return jsonify('News')
    elif result_predicted == 1 :
        return jsonify('Bola')
    elif result_predicted == 2 :
        return jsonify('Bisnis')
    elif result_predicted == 3 :
        return jsonify('Otomotif')
    elif result_predicted == 4 :
        return jsonify('Tekno')
    

if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
